In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Asegúrate de cargar el archivo 'caracteristicas_vinos.csv' en tu entorno de Google Colab
df = pd.read_csv('data_evaluacion.csv', header=None)
df.head()


In [ ]:
# Nuevas etiquetas para las columnas
nuevas_columnas = ['edad', 'clase_trabajo', 'fnlwgt', 'educacion', 'educacion_num', 'estado_civil','ocupacion', 'relacion', 
                   'raza', 'genero', 'ganancia-capital', 'perdida-capital', 'horas_semana', 'pais', 'salario']

# Asignar nuevas etiquetas
df.columns = nuevas_columnas
# Restablece el índice
df.reset_index(drop=True, inplace=True)


In [ ]:
# Reemplazar los valores '?' por '0'
df.replace('?', 0, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Elimina los espacios en blanco de los valores en la columna 'salary'
df['salario'] = df['salario'].str.strip()
salary_map = {'<=50K': 1, '>50K': 0}
df['salario'] = df['salario'].map(salary_map)


In [ ]:
df.head(10)

In [ ]:
# Elimina los espacios en blanco de los valores en la columna 'salary'
df['genero'] = df['genero'].str.strip()
# Mapea los valores
df['genero'] = df['genero'].map({'Male': 1, 'Female': 0})
print(df.head())

In [ ]:
df.head(10)

In [ ]:
# Contar los registros de cada valor en la columna 'country'
country_counts = df['pais'].value_counts()
print(country_counts)


In [ ]:
# Elimina los espacios en blanco de los valores en la columna 'salary'
df['pais'] = df['pais'].str.strip()
data = [df]
for dataset in data:
    dataset.loc[dataset['pais'] != 'United-States', 'pais'] = 'Non-US'
    dataset.loc[dataset['pais'] == 'United-States', 'pais'] = 'US'

df.head(10)


In [ ]:
df['pais'] = df['pais'].map({'US':1,'Non-US':0}).astype(int)
df.head(10)


In [ ]:
#Remplazamos estados civiles a Single y Couple
df['estado_civil'] = df['estado_civil'].str.strip()
df['estado_civil'] = df['estado_civil'].replace(['Divorced','Married-spouse-absent','Never-married','Separated','Widowed'],'Single')
df['estado_civil'] = df['estado_civil'].replace(['Married-AF-spouse','Married-civ-spouse'],'Couple')
df.head(10)


In [ ]:
df['estado_civil'] = df['estado_civil'].map({'Couple':0,'Single':1}) 
df.head(10)

In [ ]:
label_encoder = LabelEncoder()
df['relacion'] = label_encoder.fit_transform(df['relacion'])
df.head(10)

In [ ]:
# Obtener las categorías originales
categories = label_encoder.classes_

# Crear un DataFrame para mostrar el mapeo
mapping_df = pd.DataFrame({
    'Categoría': categories,
    'Código': range(len(categories))
})

print(mapping_df)

In [ ]:
label_encoder = LabelEncoder()
df['raza'] = label_encoder.fit_transform(df['raza'])
df.head(10)

In [ ]:
# Obtener las categorías originales
categories = label_encoder.classes_

# Crear un DataFrame para mostrar el mapeo
mapping_df = pd.DataFrame({
    'Categoría': categories,
    'Código': range(len(categories))
})

print(mapping_df)


In [ ]:
def definir_tipo_empleo(x):
    if x['clase_trabajo'] == 'Federal-gov' or x['clase_trabajo']== 'Local-gov' or x['clase_trabajo']=='State-gov': return 'govt'
    elif x['clase_trabajo'] == 'Private':return 'private'
    elif x['clase_trabajo'] == 'Self-emp-inc' or x['clase_trabajo'] == 'Self-emp-not-inc': return 'self_employed'
    else: return 'without_pay'
 
df['clase_trabajo'] = df['clase_trabajo'].str.strip()  
df['tipo_empleo']=df.apply(definir_tipo_empleo, axis=1)

df.head(10)

In [ ]:
label_encoder = LabelEncoder()
df['tipo_empleo'] = label_encoder.fit_transform(df['tipo_empleo'])
df.head(10)

In [ ]:
# Obtener las categorías originales
categories = label_encoder.classes_

# Crear un DataFrame para mostrar el mapeo
mapping_df = pd.DataFrame({
    'Categoría': categories,
    'Código': range(len(categories))
})

print(mapping_df)

In [ ]:
df_new = df.drop(columns=['clase_trabajo', 'educacion', 'ocupacion'])
df_new.head()

In [ ]:
df_new = df_new['edad', 'fnlwgt', 'educacion_num', 'estado_civil', 'relacion','raza', 'genero', 'ganancia-capital', 'perdida-capital',
                'horas_semana', 'pais', 'tipo_empleo', 'salario']
df_new.head()